# 1. Transforming the data from the Wiki page into a pandas dataframe

Initially, we import the necessary libraries.

In [2]:
import pandas as pd
import requests

In [8]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          61 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    beautifulsoup4-4.8.0       |           py36_0         147 KB  anaconda
    certifi-2019.6.16          |           py36_1         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0      a

In [9]:
from bs4 import BeautifulSoup

Now, we get the data from the wiki page by using BeautifulSoup library

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_text = requests.get(url).text
Bs = BeautifulSoup(wiki_text, 'html.parser')

Converting the content of Postal Code HTML table as list of data, we get:

In [11]:
data = []
for i in Bs.tbody.find_all('tr'):
    data.append([td.get_text().strip() for td in i.find_all('td')])

Creating the dataframe based on the above executions, we get:


In [12]:
df = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Since we do not require the row 0, we drop it

In [13]:
df = df.dropna()

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Now, to customize the data frame according to our ease of analysis, we do not need any row that contains a 'Not Assigned' value

In [16]:
df = df[(df.PostalCode != 'Not assigned') & (df.Borough != 'Not assigned') & (df.Neighborhood != 'Not assigned')]
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Having obtained a dataframe void of any 'Not assigned' value, we then group the dataframe by 'PostalCode' and 'Borough' and create a new dataframe containing only the said values.

In [17]:
def n_list(grouped):
    return ' ; '.join(sorted(grouped['Neighborhood'].tolist()))

new = df.groupby(['PostalCode', 'Borough'])
df_2 = new.apply(n_list).reset_index(name = 'Neighborhood')

df_2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern ; Rouge
1,M1C,Scarborough,Highland Creek ; Port Union ; Rouge Hill
2,M1E,Scarborough,Guildwood ; Morningside ; West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Testing the above table for the sorted values

In [20]:
df_2[df_2.Borough == 'North York']

,PostalCode,Borough,Neighborhood
17,M2H,North York,Hillcrest Village
18,M2J,North York,Fairview ; Henry Farm ; Oriole
19,M2K,North York,Bayview Village
20,M2L,North York,Silver Hills ; York Mills
21,M2M,North York,Newtonbrook ; Willowdale
22,M2N,North York,Willowdale South
23,M2P,North York,York Mills West
24,M2R,North York,Willowdale West
25,M3A,North York,Parkwoods
26,M3B,North York,Don Mills North


Now we save the dataframe to a csv file for future references

In [21]:
df_2.to_csv('first_task.csv', index = False)

Shape of the dataframe is given by:

In [23]:
df_2.shape

(102, 3)